# PyGSTi CHP Object Test

In [1]:
from __future__ import print_function #python 2 & 3 compatibility
import pygsti
pygsti.__version__

'0.9.9.2.post715+gcde6e292.d20210315'

In [2]:
from pygsti.objects.operation import CHPOp, StochasticCHPOp
from pygsti.tools.internalgates import standard_gatenames_chp_conversions

## CHPOp

In [3]:
c = CHPOp(['h 0', 'c 1 2'], 3)

In [4]:
print(c)
print(c.get_chp_str())
print(c.get_chp_str([3,4,5]))

CHP operation with labels: h 0, c 1 2

h 0
c 1 2

h 3
c 4 5



In [5]:
Gx = CHPOp('Gxpi 0', 1)
print(Gx)
print(Gx.get_chp_str())

CHP operation with labels: h 0, p 0, p 0, h 0

h 0
p 0
p 0
h 0



In [6]:
print(CHPOp('Gc20 0', 1))

CHP operation with labels: p 0, h 0, p 0, p 0, h 0



In [7]:
sc = StochasticCHPOp(['Gxpi 0', 'Gc20 0'], [0.2, 0.8], nqubits=1, seed_or_state=2021)
print(sc)
print(sc.get_chp_str([1]))
print(sc.get_chp_str([2]))
print(sc.get_chp_str([3]))
print(sc.get_chp_str([4]))

Stochastic CHP operation with probabilities and labels:
0.2: h 0, p 0, p 0, h 0
0.8: p 0, h 0, p 0, p 0, h 0

p 1
h 1
p 1
p 1
h 1

p 2
h 2
p 2
p 2
h 2

h 3
p 3
p 3
h 3

p 4
h 4
p 4
p 4
h 4



## CHPForwardSimulator + Explicit Model

In [8]:
chpexe = '/Users/sserita/Documents/notebooks/pyGSTi/2021-CHP/chp'
sim = pygsti.obj.CHPForwardSimulator(chpexe, shots=100)

In [9]:
#Initialize an empty Model object
model = pygsti.objects.ExplicitOpModel(['Q0', 'Q1'], simulator=sim, evotype='chp')

#Populate the Model object with states, effects, gates,
model['rho0'] = CHPOp([], nqubits=2)
model['Mdefault'] = pygsti.obj.ComputationalBasisPOVM(2, 'chp')

model['Gii'] = CHPOp([], nqubits=2)
model['Gxi'] = CHPOp(['Gxpi 0'], nqubits=2)
model['Gix'] = CHPOp(['Gxpi 1'], nqubits=2)
model['Gyi'] = CHPOp(['Gypi 0'], nqubits=2)
model['Giy'] = CHPOp(['Gypi 1'], nqubits=2)

print(model)

rho0 = CHP operation with labels: 


Mdefault = Computational(Z)-basis POVM on 2 qubits and filter None


Gii = 
CHP operation with labels: 


Gxi = 
CHP operation with labels: h 0, p 0, p 0, h 0


Gix = 
CHP operation with labels: h 1, p 1, p 1, h 1


Gyi = 
CHP operation with labels: p 0, p 0, h 0, p 0, p 0, h 0


Giy = 
CHP operation with labels: p 1, p 1, h 1, p 1, p 1, h 1






In [10]:
circ = pygsti.obj.Circuit(['Gix'])
model.probabilities(circ)

OutcomeLabelDict([(('00',), 0.0),
                  (('01',), 1.0),
                  (('10',), 0.0),
                  (('11',), 0.0)])

In [11]:
circ = pygsti.obj.Circuit(['Gix', 'Gxi'])
model.probabilities(circ)

OutcomeLabelDict([(('00',), 0.0),
                  (('01',), 0.0),
                  (('10',), 0.0),
                  (('11',), 1.0)])

In [12]:
circ = pygsti.obj.Circuit(['rho0', 'Gix', 'Gxi', 'Mdefault'])
model.probabilities(circ)

OutcomeLabelDict([(('00',), 0.0),
                  (('01',), 0.0),
                  (('10',), 0.0),
                  (('11',), 1.0)])

## ComposedOp + EmbeddedOp

In [13]:
from pygsti.objects.operation import ComposedOp, EmbeddedOp
from pygsti.objects.labeldicts import StateSpaceLabels

In [15]:
# ComposedOp
Gxx_composed = ComposedOp([CHPOp('Gxpi 0', nqubits=2), CHPOp('Gxpi 1', nqubits=2)])
print(Gxx_composed)
print(Gxx_composed.get_chp_str())

Composed operation of 2 factors:
Factor 0:
CHP operation with labels: h 0, p 0, p 0, h 0
Factor 1:
CHP operation with labels: h 1, p 1, p 1, h 1

h 0
p 0
p 0
h 0
h 1
p 1
p 1
h 1



In [16]:
# EmbeddedOp
Gxi_embedded = EmbeddedOp(['Q0', 'Q1'], ['Q0'], CHPOp('Gxpi 0', nqubits=1))
print(Gxi_embedded)
print(Gxi_embedded.get_chp_str())

Embedded operation with full dimension 4 and state space Q0(2)*Q1(2)
 that embeds the following 2-dimensional operation into acting on the ['Q0'] space
CHP operation with labels: h 0, p 0, p 0, h 0

h 0
p 0
p 0
h 0



In [17]:
Gix_embedded = EmbeddedOp(['Q0', 'Q1'], ['Q1'], CHPOp('Gxpi 0', nqubits=1))
print(Gix_embedded)
print(Gix_embedded.get_chp_str())

Embedded operation with full dimension 4 and state space Q0(2)*Q1(2)
 that embeds the following 2-dimensional operation into acting on the ['Q1'] space
CHP operation with labels: h 0, p 0, p 0, h 0

h 1
p 1
p 1
h 1



In [19]:
# Also see if I retain flexibility of targets
print(Gxi_embedded.get_chp_str([5,7]))
print(Gix_embedded.get_chp_str([5,7]))

h 5
p 5
p 5
h 5

h 7
p 7
p 7
h 7



In [25]:
# EmbeddedOp made of ComposedOps
Gxx_comp_embed = EmbeddedOp(['Q0', 'Q1', 'Q2', 'Q3'], ['Q1', 'Q2'], Gxx_composed)
print(Gxx_comp_embed)
print(Gxx_comp_embed.get_chp_str())
print(Gxx_comp_embed.get_chp_str([5, 6, 7, 8]))

Embedded operation with full dimension 16 and state space Q0(2)*Q1(2)*Q2(2)*Q3(2)
 that embeds the following 4-dimensional operation into acting on the ['Q1', 'Q2'] space
Composed operation of 2 factors:
Factor 0:
CHP operation with labels: h 0, p 0, p 0, h 0
Factor 1:
CHP operation with labels: h 1, p 1, p 1, h 1

h 1
p 1
p 1
h 1
h 2
p 2
p 2
h 2

h 6
p 6
p 6
h 6
h 7
p 7
p 7
h 7

